# **Assignment 3**
# **LSTM - IMDB, Spam, Tweet emotions, Cellphone Ratings**
## Parameshwari S


In [1]:
import numpy as np
import pandas as pd
from keras.datasets import imdb
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Embedding
#from tensorflow.keras.optimizers import RMSprop
#from keras.layers.embeddings import Embedding
#from keras.preprocessing import sequence
#from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.utils import to_categorical
#from keras.callbacks import EarlyStopping
from tensorflow.keras import layers
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import spacy
nlp = spacy.load('en_core_web_sm')
#from sklearn.preprocessing import LabelEncoder
#from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## IMDB

In [2]:
np.random.seed(7)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = 6000)

17473536/17464789 [==============================] - 0s 0us/step


In [4]:
x_train = pad_sequences(x_train, maxlen = 500)
x_test = pad_sequences(x_test, maxlen = 500)

In [5]:
model = Sequential()
model.add(Embedding(6000, 32, input_length = 500))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           192000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 245,301
Trainable params: 245,301
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(x_train, y_train, epochs = 3, batch_size = 64, verbose = True)
loss, accuracy = model.evaluate(x_test, y_test)
print("Accuracy - ", accuracy)

Epoch 1/3
391/391 [==============================] - 217s 549ms/step - loss: 0.4543 - accuracy: 0.7842
Epoch 2/3
391/391 [==============================] - 213s 545ms/step - loss: 0.2856 - accuracy: 0.8860
Epoch 3/3
782/782 [==============================] - 69s 88ms/step - loss: 0.3117 - accuracy: 0.8723
Accuracy -  0.8722800016403198


## Spam classifier

In [ ]:
df = pd.read_csv("/content/drive/My Drive/NLP/spam.csv", usecols = ['v1', 'v2'], encoding = 'latin-1')
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.shape

(5572, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
df['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [ ]:
x=df.v2
y=df.v1
le=LabelEncoder()
y=le.fit_transform(y)
y=y.reshape(-1,1)

In [ ]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.15)

In [ ]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(x_train)
sequences = tok.texts_to_sequences(x_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(), metrics = ['accuracy'])

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_15 (Embedding)    (None, 150, 50)           50000     
                                                                 
 lstm_15 (LSTM)              (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_27 (Activation)  (None, 256)               0         
                                                                 
 dropout_13 (Dropout)        (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257

In [ ]:
model.fit(sequences_matrix, y_train, batch_size = 128, epochs = 10, validation_split = 0.2, callbacks = [EarlyStopping(monitor='val_loss',min_delta=0.0001)])
#min_delta - min loss between two epochs

Epoch 1/10
30/30 [==============================] - 9s 213ms/step - loss: 0.3386 - accuracy: 0.8712 - val_loss: 0.1460 - val_accuracy: 0.9747
Epoch 2/10
30/30 [==============================] - 6s 193ms/step - loss: 0.0892 - accuracy: 0.9791 - val_loss: 0.0604 - val_accuracy: 0.9842


In [ ]:
test_sequences = tok.texts_to_sequences(x_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
loss, accuracy = model.evaluate(test_sequences_matrix, y_test)
print("Accuracy - ", accuracy*100)

27/27 [==============================] - 1s 20ms/step - loss: 0.0657 - accuracy: 0.9856
Accuracy -  98.56459498405457


## Tweet emotion classification

In [ ]:
df1 = pd.read_csv("/content/drive/My Drive/NLP/judge-1377884607_tweet_product_company.csv", encoding = 'latin-1')
df1.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [ ]:
df1.rename(columns = {"is_there_an_emotion_directed_at_a_brand_or_product": "emotions"}, inplace = True)

In [ ]:
df1.drop('emotion_in_tweet_is_directed_at', axis = 1, inplace = True)
df1.head()

,tweet_text,emotions
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


In [ ]:
df.shape

(5572, 2)

In [ ]:
df1.dtypes

tweet_text    object
emotions      object
dtype: object

In [ ]:
df1['tweet_text'][245]

'Brilliant and enlightening game mechanics presentation by @mention from Google #SXSW'

In [ ]:
def remove_at(text):
  text = str(text).lower()
  text = re.sub(r'@\S+', ' ', text)
  return text

def remove_hash(text):
  text = re.sub(r'#\S+', ' ', text)
  return text

def remove_urls(text):
  text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
  return text

def remove_special(text):
  text = re.sub(r'[^a-zA-Z0-9\s]+', ' ', text)
  return text

def remove_num(text):
  text = re.sub(r'\d+', ' ', text)
  return text

df1['tweet_text'] = df1['tweet_text'].apply(remove_at)
df1['tweet_text'] = df1['tweet_text'].apply(remove_hash)
df1['tweet_text'] = df1['tweet_text'].apply(remove_urls)
df1['tweet_text'] = df1['tweet_text'].apply(remove_special)
df1['tweet_text'] = df1['tweet_text'].apply(remove_num)

In [ ]:
lemma = WordNetLemmatizer()
stopwords = nlp.Defaults.stop_words
def clean(text):
  tokens = word_tokenize(text)
  tokens_stop = [word for word in tokens if word not in stopwords]
  lemmatized_words = [lemma.lemmatize(token) for token in tokens_stop if len(token)>2]
  clean_text = ' '.join(lemmatized_words)
  return clean_text

df1['tweet_text'] = df1['tweet_text'].apply(clean)
df1.head()

,tweet_text,emotions
0,iphone hr tweeting dead need upgrade plugin st...,Negative emotion
1,know awesome ipad iphone app likely appreciate...,Positive emotion
2,wait sale,Positive emotion
3,hope year festival isn crashy year iphone app,Negative emotion
4,great stuff fri marissa mayer google tim reill...,Positive emotion


In [ ]:
le = LabelEncoder()
df1['emotions'] = le.fit_transform(df1['emotions'])
df1.head()

,tweet_text,emotions
0,iphone hr tweeting dead need upgrade plugin st...,1
1,know awesome ipad iphone app likely appreciate...,3
2,wait sale,3
3,hope year festival isn crashy year iphone app,1
4,great stuff fri marissa mayer google tim reill...,3


In [ ]:
x = df1.tweet_text
y = df1.emotions
y = le.fit_transform(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 100)

In [ ]:
y_train = to_categorical(y_train,num_classes=4)
y_test = to_categorical(y_test,num_classes=4)

In [ ]:
max_words = 1000
max_len = 100
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(x_train)
sequences = tok.texts_to_sequences(x_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)

In [ ]:
def RNN():
  inputs = Input(name='inputs',shape=[max_len])
  layer = Embedding(max_words,256,input_length=max_len)(inputs)
  layer = LSTM(512)(layer)
  layer = Dense(256,name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(4,name='out_layer')(layer)
  layer = Activation('softmax')(layer)
  model = Model(inputs=inputs,outputs=layer)
  return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Model: "model_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 100)]             0         
                                                                 
 embedding_34 (Embedding)    (None, 100, 256)          256000    
                                                                 
 lstm_34 (LSTM)              (None, 512)               1574912   
                                                                 
 FC1 (Dense)                 (None, 256)               131328    
                                                                 
 activation_65 (Activation)  (None, 256)               0         
                                                                 
 dropout_32 (Dropout)        (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 4)                 102

In [ ]:
model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
43/43 [==============================] - 185s 4s/step - loss: 0.9816 - accuracy: 0.5679 - val_loss: 0.9066 - val_accuracy: 0.5872
Epoch 2/10
43/43 [==============================] - 172s 4s/step - loss: 0.8569 - accuracy: 0.6180 - val_loss: 0.8434 - val_accuracy: 0.6393


In [ ]:
test_sequences = tok.texts_to_sequences(x_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
loss, accuracy = model.evaluate(test_sequences_matrix, y_test)
print("Accuracy - ", accuracy*100)

72/72 [==============================] - 27s 374ms/step - loss: 0.8601 - accuracy: 0.6324
Accuracy -  63.2365882396698


## Cellphone ratings

In [ ]:
df2 = pd.read_csv("/content/drive/My Drive/NLP/CellPhonesRating.csv", usecols = ['reviewText', 'rating'])
df2.head()

,reviewText,rating
0,They look good and stick good! I just don't li...,4.0
1,These stickers work like the review says they ...,5.0
2,These are awesome and make my phone look so st...,5.0
3,Item arrived in great time and was in perfect ...,4.0
4,"awesome! stays on, and looks great. can be use...",5.0


In [ ]:
df2['rating'] = df2['rating'].apply(lambda x: 'Positive' if x > 3 else 'Negative')
df2.head()

,reviewText,rating
0,They look good and stick good! I just don't li...,Positive
1,These stickers work like the review says they ...,Positive
2,These are awesome and make my phone look so st...,Positive
3,Item arrived in great time and was in perfect ...,Positive
4,"awesome! stays on, and looks great. can be use...",Positive


In [ ]:
df2.shape

(190722, 2)

In [ ]:
df2['reviewText'][8]

"Good case, solid build. Protects phone all around with good access to buttons. Battery charges with full battery lasts me a full day. I usually leave my house around 7am and return at 10pm. I'm glad that it lasts from start to end. 5/5"

In [ ]:
def remove_special(text):
  text = str(text).lower()
  text = re.sub(r'[^a-zA-Z0-9\s]+', ' ', text)
  return text

def remove_num(text):
  text = re.sub(r'\d+', ' ', text)
  return text

df2['reviewText'] = df2['reviewText'].apply(remove_special)
df2['reviewText'] = df2['reviewText'].apply(remove_num)

In [ ]:
lemma = WordNetLemmatizer()
stopwords = nlp.Defaults.stop_words
def clean(text):
  tokens = word_tokenize(text)
  tokens_stop = [word for word in tokens if word not in stopwords]
  lemmatized_words = [lemma.lemmatize(token) for token in tokens_stop if len(token)>2]
  clean_text = ' '.join(lemmatized_words)
  return clean_text

df2['reviewText'] = df2['reviewText'].apply(clean)
df2.head()

,reviewText,rating
0,look good stick good don like rounded shape bu...,Positive
1,sticker work like review say stick great stay ...,Positive
2,awesome phone look stylish far year believe ye...,Positive
3,item arrived great time perfect condition orde...,Positive
4,awesome stay look great multiple apple product...,Positive


In [ ]:
le = LabelEncoder()
y = le.fit_transform(df2['rating'])
y = y.reshape(-1,1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df2['reviewText'], y, test_size = 0.25, random_state = 100)

In [ ]:
max_words = 5000
max_len = 60
tok = Tokenizer(max_words)
tok.fit_on_texts(x_train)
sequences = tok.texts_to_sequences(x_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
def RNN():
  inputs = Input(name='inputs',shape=[max_len])
  layer = Embedding(max_words,50,input_length=max_len)(inputs)
  layer = LSTM(64)(layer)
  layer = Dense(256,name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(1,name='out_layer')(layer)
  layer = Activation('sigmoid')(layer)
  model = Model(inputs=inputs,outputs=layer)
  return model

In [ ]:
model = RNN()
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(sequences_matrix,y_train,batch_size = 128, epochs=10,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
894/894 [==============================] - 83s 90ms/step - loss: 0.3802 - accuracy: 0.8346 - val_loss: 0.3455 - val_accuracy: 0.8477
Epoch 2/10
894/894 [==============================] - 78s 87ms/step - loss: 0.3297 - accuracy: 0.8582 - val_loss: 0.3432 - val_accuracy: 0.8506


In [ ]:
test_sequences = tok.texts_to_sequences(x_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
loss, accuracy = model.evaluate(test_sequences_matrix, y_test)
print("Accuracy - ", accuracy*100)

1491/1491 [==============================] - 16s 11ms/step - loss: 0.5325 - accuracy: 0.8439
Accuracy -  84.39210653305054
